In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.ar_model import AR
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.stattools import adfuller  

%matplotlib inline

import warnings
warnings.simplefilter(action="ignore")

import sqlite3

C:\Users\ng_li\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
order = pd.read_csv('./orders_data.csv')
order.head()

,Created Date,Country,City,Restaurant ID,Restaurant Name,Order State,Cancel Reason,Cuisine,Platform,Products in Order,Order Value € (Gross),Delivery Fee,Delivery Time,Order ID
0,28.02.2020,Portugal,Lisbon,7238,Chirashi - Alvalade,delivered,NaN,Sushi,ios,13,€89.30,0.0,17.7,1381
1,28.02.2020,Portugal,Lisbon,12758,Istanbul Kebab Pizza - Alameda,delivered,NaN,Pizza,android,1,€10.00,0.0,16.3,2382
2,28.02.2020,Portugal,Lisbon,6631,A-100 - Lisboa,delivered,NaN,Burgers,ios,2,€16.96,0.0,13.6,3383
3,28.02.2020,Portugal,Lisbon,10535,La Paparrucha - Parrilla Argentina,delivered,NaN,South American,ios,3,€25.15,4.2,26.1,4384
4,28.02.2020,Portugal,Lisbon,9695,Choupana Caffe,delivered,NaN,Brunch,ios,3,€16.00,0.0,11.6,5385


In [3]:
order.columns = order.columns.str.replace('€ (Gross)', 'Gross', regex=False)

In [4]:
# change column name to lowercase and replace ' ' with '_'
order.columns = order.columns.str.replace(' ', '_', regex=False).str.lower()

In [5]:
order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99618 entries, 0 to 99617
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   created_date       99618 non-null  object 
 1   country            99618 non-null  object 
 2   city               99618 non-null  object 
 3   restaurant_id      99618 non-null  int64  
 4   restaurant_name    99618 non-null  object 
 5   order_state        99618 non-null  object 
 6   cancel_reason      1682 non-null   object 
 7   cuisine            99291 non-null  object 
 8   platform           87684 non-null  object 
 9   products_in_order  99618 non-null  int64  
 10  order_value_gross  97936 non-null  object 
 11  delivery_fee       97936 non-null  float64
 12  delivery_time      97936 non-null  float64
 13  order_id           99618 non-null  int64  
dtypes: float64(2), int64(3), object(9)
memory usage: 10.6+ MB


In [6]:
order["created_date"]= pd.to_datetime(order["created_date"], format="%d.%m.%Y", infer_datetime_format=True)

In [7]:
order["order_value_gross"] = order["order_value_gross"].str.strip('€')

In [8]:
order["order_value_gross"] = order["order_value_gross"].astype(float)

In [9]:
order.head()

,created_date,country,city,restaurant_id,restaurant_name,order_state,cancel_reason,cuisine,platform,products_in_order,order_value_gross,delivery_fee,delivery_time,order_id
0,2020-02-28,Portugal,Lisbon,7238,Chirashi - Alvalade,delivered,NaN,Sushi,ios,13,89.30,0.0,17.7,1381
1,2020-02-28,Portugal,Lisbon,12758,Istanbul Kebab Pizza - Alameda,delivered,NaN,Pizza,android,1,10.00,0.0,16.3,2382
2,2020-02-28,Portugal,Lisbon,6631,A-100 - Lisboa,delivered,NaN,Burgers,ios,2,16.96,0.0,13.6,3383
3,2020-02-28,Portugal,Lisbon,10535,La Paparrucha - Parrilla Argentina,delivered,NaN,South American,ios,3,25.15,4.2,26.1,4384
4,2020-02-28,Portugal,Lisbon,9695,Choupana Caffe,delivered,NaN,Brunch,ios,3,16.00,0.0,11.6,5385


In [10]:
order.isnull().sum()

created_date             0
country                  0
city                     0
restaurant_id            0
restaurant_name          0
order_state              0
cancel_reason        97936
cuisine                327
platform             11934
products_in_order        0
order_value_gross     1682
delivery_fee          1682
delivery_time         1682
order_id                 0
dtype: int64

In [11]:
order['order_state'].unique()

array(['delivered', 'failed', 'rejected'], dtype=object)

In [12]:
order[order['order_state']=='delivered'].isnull().sum()

created_date             0
country                  0
city                     0
restaurant_id            0
restaurant_name          0
order_state              0
cancel_reason        97936
cuisine                323
platform             11707
products_in_order        0
order_value_gross        0
delivery_fee             0
delivery_time            0
order_id                 0
dtype: int64

In [13]:
order[(order['order_state']=='failed')|(order['order_state']=='rejected')].isnull().sum()

created_date            0
country                 0
city                    0
restaurant_id           0
restaurant_name         0
order_state             0
cancel_reason           0
cuisine                 4
platform              227
products_in_order       0
order_value_gross    1682
delivery_fee         1682
delivery_time        1682
order_id                0
dtype: int64

In [14]:
# if dtype = object, fill NaN with 'NA'
# if dtype = int or float, fill NaN with 0
for col in order:
    #get dtype for column
    dt = order[col].dtype 
    #check if it is a number
    if dt == int or dt == float:
        order[col].fillna(0, inplace=True)
    else:
        order[col].fillna('NA', inplace=True)

In [15]:
order.isnull().sum()

created_date         0
country              0
city                 0
restaurant_id        0
restaurant_name      0
order_state          0
cancel_reason        0
cuisine              0
platform             0
products_in_order    0
order_value_gross    0
delivery_fee         0
delivery_time        0
order_id             0
dtype: int64

In [16]:
payment = pd.read_csv('./orders_payments_info.csv')
payment.head()

,Order ID,Payment Method,Card Issuer
0,1381,cashless,BANKINTER S.A.
1,2382,cashless,BANCO COMERCIAL PORTUGUES S.A.
2,3383,cashless,BANCO COMERCIAL PORTUGUES S.A.
3,4384,cashless,ING BANK N.V.
4,5385,cashless,INTESA SANPAOLO SPA


In [17]:
# change column name to lowercase and replace ' ' with '_'
payment.columns = payment.columns.str.replace(' ', '_', regex=False).str.lower()

In [18]:
payment.isnull().sum()

order_id              0
payment_method        0
card_issuer       12712
dtype: int64

In [19]:
payment['payment_method'].unique()

array(['cashless', 'cash'], dtype=object)

In [20]:
payment[payment['payment_method']=='cash'].isnull().sum()

order_id             0
payment_method       0
card_issuer       5761
dtype: int64

In [21]:
payment['payment_method'].fillna('NA', inplace=True)

In [22]:
payment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99618 entries, 0 to 99617
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   order_id        99618 non-null  int64 
 1   payment_method  99618 non-null  object
 2   card_issuer     86906 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.3+ MB


In [23]:
conn = sqlite3.connect("data.db")

In [24]:
c = conn.cursor()
c.execute('''DROP TABLE IF EXISTS [orders,payment]''')
order.to_sql('orders', conn, if_exists='replace', index=False)

99618

c.execute('''DROP TABLE IF EXISTS orders''')
payment.to_sql('payment', conn, if_exists='replace', index=False)

conn.row_factory = sqlite3.Row

In [25]:
def execute_sql(sql):
    """
    Functions to execute results and return as dataframe
    """
   
    result = pd.read_sql(sql, conn)
    return pd.DataFrame(result)

In [26]:
sql = """SELECT created_date FROM orders limit 5"""
execute_sql(sql)

,created_date
0,2020-02-28 00:00:00
1,2020-02-28 00:00:00
2,2020-02-28 00:00:00
3,2020-02-28 00:00:00
4,2020-02-28 00:00:00


In [27]:
c.execute("SELECT *\
from orders limit 5")
          
          
rows = c.fetchall()
rows

[('2020-02-28 00:00:00',
  'Portugal',
  'Lisbon',
  7238,
  'Chirashi - Alvalade',
  'delivered',
  'NA',
  'Sushi',
  'ios',
  13,
  89.3,
  0.0,
  17.7,
  1381),
 ('2020-02-28 00:00:00',
  'Portugal',
  'Lisbon',
  12758,
  'Istanbul Kebab Pizza - Alameda',
  'delivered',
  'NA',
  'Pizza',
  'android',
  1,
  10.0,
  0.0,
  16.3,
  2382),
 ('2020-02-28 00:00:00',
  'Portugal',
  'Lisbon',
  6631,
  'A-100 - Lisboa',
  'delivered',
  'NA',
  'Burgers',
  'ios',
  2,
  16.96,
  0.0,
  13.6,
  3383),
 ('2020-02-28 00:00:00',
  'Portugal',
  'Lisbon',
  10535,
  'La Paparrucha - Parrilla Argentina',
  'delivered',
  'NA',
  'South American',
  'ios',
  3,
  25.15,
  4.2,
  26.1,
  4384),
 ('2020-02-28 00:00:00',
  'Portugal',
  'Lisbon',
  9695,
  'Choupana Caffe',
  'delivered',
  'NA',
  'Brunch',
  'ios',
  3,
  16.0,
  0.0,
  11.6,
  5385)]

In [28]:
c.execute("SELECT *\
from payment limit 5")
          
          
rows = c.fetchall()
rows

[(1381, 'cashless', 'BANKINTER S.A.'),
 (2382, 'cashless', 'BANCO COMERCIAL PORTUGUES S.A.'),
 (3383, 'cashless', 'BANCO COMERCIAL PORTUGUES S.A.'),
 (4384, 'cashless', 'ING BANK N.V.'),
 (5385, 'cashless', 'INTESA SANPAOLO SPA')]

In [29]:
sql = """
SELECT 
    Order_ID, CASE CAST (strftime('%w',
        created_date) AS integer)
      WHEN 0 THEN 'Sunday'
      WHEN 1 THEN 'Monday'
      WHEN 2 THEN 'Tuesday'
      WHEN 3 THEN 'Wednesday'
      WHEN 4 THEN 'Thursday'
      WHEN 5 THEN 'Friday'
    ELSE
    'Saturday'
  END
    AS weekday,
    Payment_Method 
FROM orders 
    LEFT JOIN orders openOrderParts ON openOrderParts.order_number = openOrderParts.order_number 
        AND openOrderParts.date_deleted IS NULL
WHERE date_deleted IS NOT NULL AND openOrderParts IS NULL
GROUP BY order_number

SyntaxError: EOF while scanning triple-quoted string literal (181921276.py, line 20)

In [ ]:
sql = """
  SELECT
  o.Order_ID,
    strftime('%W',
      o.created_date) week,
    CASE CAST (strftime('%w',
        o.created_date) AS integer)
      WHEN 0 THEN 'Sunday'
      WHEN 1 THEN 'Monday'
      WHEN 2 THEN 'Tuesday'
      WHEN 3 THEN 'Wednesday'
      WHEN 4 THEN 'Thursday'
      WHEN 5 THEN 'Friday'
    ELSE
    'Saturday'
  END
    AS weekday,
    o.City,
    Payment_Method,
    o.Order_State,
    COUNT(o.Order_ID) count
  FROM
    orders o
  JOIN
    payment p
  ON
    o.Order_ID = p.Order_ID
    LEFT JOIN orders a ON o.Order_ID = a.Order_ID 
        AND a.count IS NULL
WHERE count IS NOT NULL AND a IS NULL

  and 
    Order_State = "delivered"
  GROUP BY
    week,
    weekday,
    City,
    Payment_Method,o.Order_ID
    
    
"""

execute_sql(sql)

In [ ]:
sql = """
SELECT
strftime('%W',
      created_date) week,
      city,
   CASE CAST (strftime('%w',
        created_date) AS integer)
      WHEN 0 THEN 'Sunday'
      WHEN 1 THEN 'Monday'
      WHEN 2 THEN 'Tuesday'
      WHEN 3 THEN 'Wednesday'
      WHEN 4 THEN 'Thursday'
      WHEN 5 THEN 'Friday'
    ELSE
    'Saturday'
  END
    AS weekday,
    Payment_Method,
    CASE WHEN COUNT(*) = COUNT(Payment_Method) THEN COUNT(Payment_Method) END AS date_deleted
  FROM
    orders o
  JOIN
    payment p
  ON
    o.Order_ID = p.Order_ID
  WHERE
    Order_State = "delivered" 
  GROUP BY
  week,
  city,
    weekday,
    Payment_Method
    """

execute_sql(sql)

In [ ]:
SELECT orders.city, orders.week, payments.payment_method, orders.weekday_name, COUNT(orders.order_id) as order_count
FROM order_table as orders
INNER JOIN payment_table as payments ON orders.order_id = payments.order_id
GROUP BY orders.city, orders.week, payments.payment_method, orders.weekday_name
HAVING order_count = (
    SELECT MAX(order_count) FROM (
        SELECT COUNT(orders.order_id) as order_count
        FROM order_table as orders
        INNER JOIN payment_table as payments ON orders.order_id = payments.order_id
        WHERE orders.city = orders.city AND orders.week = orders.week AND payments.payment_method = payments.payment_method
        GROUP BY orders.weekday_name
    ) as t
)


This query first joins the order_table and payment_table on the order_id column, then groups the results by city, week, payment_method, and weekday_name. It then uses a subquery to find the maximum order count for each payment method, week, and city, and filters the results to only include rows where the order count is equal to the maximum order count. If there were no cash orders during the week in a city, the query will return NULL for the weekday with the highest order count. If several days during the week have the same count of delivered orders per payment method, the query will return the weekday name of an earlier day (Sunday is considered the first day of the week with an index of 0).

In [ ]:
SELECT orders.city, DATEPART(wk, CONVERT(date, orders.created_date, 105)) as week, payments.payment_method, DATENAME(dw, CONVERT(date, orders.created_date, 105)) as weekday_name, COUNT(orders.order_id) as order_count
FROM order_table as orders
INNER JOIN payment_table as payments ON orders.order_id = payments.order_id
GROUP BY orders.city, DATEPART(wk, CONVERT(date, orders.created_date, 105)), payments.payment_method, DATENAME(dw, CONVERT(date, orders.created_date, 105))
HAVING order_count = (
    SELECT MAX(order_count) FROM (
        SELECT COUNT(orders.order_id) as order_count
        FROM order_table as orders
        INNER JOIN payment_table as payments ON orders.order_id = payments.order_id
        WHERE orders.city = orders.city AND DATEPART(wk, CONVERT(date, orders.created_date, 105)) = DATEPART(wk, CONVERT(date, orders.created_date, 105)) AND payments.payment_method = payments.payment_method
        GROUP BY DATENAME(dw, CONVERT(date, orders.created_date, 105))
    ) as t
)


This query uses the CONVERT function to convert the created_date string to a date data type, using the format code 105 to specify the "dd.mm.yyyy" format. It then uses the DATEPART function to extract the week from the converted date, and the DATENAME function to extract the weekday name. It then groups the results by city, week, payment_method, and weekday_name, and uses a subquery to find the maximum order count for each payment method, week, and city. If there were no cash orders during the week in a city, the query will return NULL for the weekday with the highest order count. If several days during the week have the same count of delivered orders per payment method, the query will return the weekday name of an earlier day (Sunday is considered the first day of the week with an index of 0).

In [52]:
sql = """
SELECT
o.city, strftime('%W', o.created_date) week,
p.payment_method,
CASE CAST (strftime('%w',
        o.created_date) AS integer)
      WHEN 0 THEN 'Sunday'
      WHEN 1 THEN 'Monday'
      WHEN 2 THEN 'Tuesday'
      WHEN 3 THEN 'Wednesday'
      WHEN 4 THEN 'Thursday'
      WHEN 5 THEN 'Friday'
    ELSE
    'Saturday'
  END
    AS weekday,
COUNT(o.order_id) order_count
FROM orders o
INNER JOIN payment p
ON o.order_id = p.order_id
GROUP BY o.city, week, p.payment_method, weekday
HAVING order_count = (
    SELECT MAX(order_count) FROM (
        SELECT COUNT(o.order_id) as order_count, p.payment_method, strftime('%W', o.created_date) week 
        FROM orders o
        Left JOIN payment p
        ON o.order_id = p.order_id
        WHERE o.city = o.city
        AND strftime('%W', o.created_date) = week
        GROUP BY o.city, week, p.payment_method
    ) as t
    WHERE p.payment_method = t.payment_method
)

"""

execute_sql(sql)

,city,week,payment_method,weekday,order_count


In [58]:
sql = """
SELECT
strftime('%W', created_date) week,
o.city, 
p.payment_method,
CASE CAST (strftime('%w',
        created_date) AS integer)
      WHEN 0 THEN 'Sunday'
      WHEN 1 THEN 'Monday'
      WHEN 2 THEN 'Tuesday'
      WHEN 3 THEN 'Wednesday'
      WHEN 4 THEN 'Thursday'
      WHEN 5 THEN 'Friday'
    ELSE
    'Saturday'
  END
    AS weekday,
COUNT(o.order_id) order_count
FROM orders o
INNER JOIN payment p
ON o.order_id = p.order_id
GROUP BY o.city, week, p.payment_method, weekday
HAVING order_count = (
    SELECT MAX(order_count) FROM (
        SELECT COUNT(o.order_id) as order_count,
        CASE CAST (strftime('%w',
        created_date) AS integer)
      WHEN 0 THEN 'Sunday'
      WHEN 1 THEN 'Monday'
      WHEN 2 THEN 'Tuesday'
      WHEN 3 THEN 'Wednesday'
      WHEN 4 THEN 'Thursday'
      WHEN 5 THEN 'Friday'
    ELSE
    'Saturday'
  END AS weekday
        FROM orders o
        Left JOIN payment p
        ON o.order_id = p.order_id
        AND order_count IS NULL
        WHERE 
        o.order_id = o.order_id
        GROUP BY o.city, p.payment_method,
        weekday,strftime('%W', created_date)
    ) as t
)


"""

execute_sql(sql)

DatabaseError: Execution failed on sql '
SELECT
strftime('%W', created_date) week,
o.city, 
p.payment_method,
CASE CAST (strftime('%w',
        created_date) AS integer)
      WHEN 0 THEN 'Sunday'
      WHEN 1 THEN 'Monday'
      WHEN 2 THEN 'Tuesday'
      WHEN 3 THEN 'Wednesday'
      WHEN 4 THEN 'Thursday'
      WHEN 5 THEN 'Friday'
    ELSE
    'Saturday'
  END
    AS weekday,
COUNT(o.order_id) order_count
FROM orders o
INNER JOIN payment p
ON o.order_id = p.order_id
GROUP BY o.city, week, p.payment_method, weekday
HAVING order_count = (
    SELECT MAX(order_count) FROM (
        SELECT COUNT(o.order_id) as order_count,
        CASE CAST (strftime('%w',
        created_date) AS integer)
      WHEN 0 THEN 'Sunday'
      WHEN 1 THEN 'Monday'
      WHEN 2 THEN 'Tuesday'
      WHEN 3 THEN 'Wednesday'
      WHEN 4 THEN 'Thursday'
      WHEN 5 THEN 'Friday'
    ELSE
    'Saturday'
  END AS weekday
        FROM orders o
        Left JOIN payment p
        ON o.order_id = p.order_id
      

The query uses the following steps to achieve the desired result:

The INNER JOIN clause is used to join the order_table and payment_table tables based on the order_id column. This allows us to access the data from both tables in a single query.

The GROUP BY clause is used to group the data by city, week, payment_method, and weekday. This will allow us to count the number of orders for each combination of these values.

The COUNT function is used to count the number of orders for each group.

The HAVING clause is used to filter the groups that have the highest number of orders. To do this, we use a subquery to find the maximum number of orders for each group and compare it to the order_count column in the outer query.

The CASE statement is used to convert the day of the week number (0-6) to the corresponding weekday name.

The STRFTIME function is used to extract the week number from the created_date column.

The SUBSTR function is used to extract the year, month, and day from the created_date column and concatenate them into a date string in the format "yyyy-mm-dd". This date string is then cast to a date using the CAST function. The DATENAME function is then used to extract the weekday name from the date.

By following these steps, the query returns the weekday names with the highest number of delivered orders for each of the payment

In [ ]:
SELECT orders.city, STRFTIME('%W', CAST(SUBSTR(orders.created_date, 7, 4) || '-' || SUBSTR(orders.created_date, 4, 2) || '-' || SUBSTR(orders.created_date, 1, 2) as date)) as week, payments.payment_method, DATENAME(dw, CAST(SUBSTR(orders.created_date, 7, 4) || '-' || SUBSTR(orders.created_date, 4, 2) || '-' || SUBSTR(orders.created_date, 1, 2) as date)) as weekday_name, COUNT(orders.order_id) as order_count
FROM order_table as orders
INNER JOIN payment_table as payments ON orders.order_id = payments.order_id
GROUP BY orders.city, STRFTIME('%W', CAST(SUBSTR(orders.created_date, 7, 4) || '-' || SUBSTR(orders.created_date, 4, 2) || '-' || SUBSTR(orders.created_date, 1, 2) as date)), payments.payment_method, DATENAME(dw, CAST(SUBSTR(orders.created_date, 7, 4) || '-' || SUBSTR(orders.created_date, 4, 2) || '-' || SUBSTR(orders.created_date, 1, 2) as date))
HAVING order_count = (
    SELECT MAX(order_count) FROM (
        SELECT COUNT(orders.order_id) as order_count
        FROM order_table as orders
        INNER JOIN payment_table as payments ON orders.order_id = payments.order_id
        WHERE orders.city = orders.city AND STRFTIME('%W', CAST(SUBSTR(orders.created_date, 7, 4) || '-' || SUBSTR(orders.created_date, 4, 2) || '-' || SUBSTR(orders.created_date, 1, 2) as date)) = STRFTIME('%W', CAST(SUBSTR(orders.created_date, 7, 4) || '-' || SUBSTR(orders.created_date, 4, 2) || '-' || SUBSTR(orders.created_date, 1, 2


In [ ]:
SELECT orders.city, DATEPART(wk, orders.created_date) as week, payments.payment_method, DATENAME(dw, orders.created_date) as weekday_name, COUNT(orders.order_id) as order_count
FROM order_table as orders
INNER JOIN payment_table as payments ON orders.order_id = payments.order_id
GROUP BY orders.city, DATEPART(wk, orders.created_date), payments.payment_method, DATENAME(dw, orders.created_date)
HAVING order_count = (
    SELECT MAX(order_count) FROM (
        SELECT COUNT(orders.order_id) as order_count
        FROM order_table as orders
        INNER JOIN payment_table as payments ON orders.order_id = payments.order_id
        WHERE orders.city = orders.city AND DATEPART(wk, orders.created_date) = DATEPART(wk, orders.created_date) AND payments.payment_method = payments.payment_method
        GROUP BY DATENAME(dw, orders.created_date)
    ) as t
)


In [74]:
sql = """
SELECT
  city,
  payment_method,
  strftime('%W', created_date) AS week,
  CASE strftime('%w', created_date)
    WHEN 0 THEN 'Sunday'
    WHEN 1 THEN 'Monday'
    WHEN 2 THEN 'Tuesday'
    WHEN 3 THEN 'Wednesday'
    WHEN 4 THEN 'Thursday'
    WHEN 5 THEN 'Friday'
    WHEN 6 THEN 'Saturday'
  END AS weekday,
  COUNT(*) AS order_count
FROM orders
JOIN payment
ON orders.order_id = payment.order_id
WHERE order_state = 'delivered'
GROUP BY city, payment_method, week, weekday
HAVING COUNT(*) = (SELECT MAX(cnt) FROM (
  SELECT COUNT(*) AS cnt
  FROM orders
  LEFT JOIN payment
  ON orders.order_id = payment.order_id
  WHERE order_state = 'delivered'
  GROUP BY city, strftime('%W', created_date)
) t)
ORDER BY city, payment_method, week, weekday;


"""

execute_sql(sql)

,city,payment_method,week,weekday,order_count
0,Lisbon,cashless,06,None,11484


In [83]:
sql = """
WITH
  table1 AS(
  SELECT
    strftime('%W',
      created_date) week,
    CAST (strftime('%w',
        created_date) AS integer) weekday_num,
    City,
    Payment_Method,
    Order_State,
    COUNT(*) count
  FROM
    orders o
  Left JOIN
    payment p
  ON
    o.Order_ID = p.Order_ID
  WHERE
    Order_State = "delivered"),
rnk_table AS (
  SELECT
    *,
    RANK() OVER (PARTITION BY city,week, Payment_Method ORDER BY count DESC ) AS rnk
  FROM
    table1 )
    SELECT
  week,
  City,
  Payment_Method,
  case
  when
  week=week
  and City = City
  and Payment_Method = Payment_Method
  and count = count
  then min(weekday_num)
  else weekday_num end weekday_num
  
FROM
  rnk_table
WHERE
  rnk = 1
ORDER BY
  week,
  City,
  Payment_Method,
  weekday_num 
  
"""

execute_sql(sql)

,week,City,Payment_Method,weekday_num
0,08,Lisbon,cashless,5


In [70]:
pd.set_option('display.max_rows', 200)

In [80]:
sql = """
WITH
  table1 AS(
  SELECT
    strftime('%W',
      created_date) week,
    CASE CAST (strftime('%w',
        created_date) AS integer)
      WHEN 0 THEN 'Sunday'
      WHEN 1 THEN 'Monday'
      WHEN 2 THEN 'Tuesday'
      WHEN 3 THEN 'Wednesday'
      WHEN 4 THEN 'Thursday'
      WHEN 5 THEN 'Friday'
    ELSE
    'Saturday'
  END
    AS weekday,
    City,
    Payment_Method,
    Order_State,
    COUNT(*) count
  FROM
    orders o
  JOIN
    payment p
  ON
    o.Order_ID = p.Order_ID
  WHERE
    Order_State = "delivered"
  GROUP BY
    week,
    weekday,
    City,
    Payment_Method)
  SELECT
    *,
    DENSE_RANK() OVER (PARTITION BY city,week, Payment_Method ORDER BY count DESC ) AS rnk
  FROM
    table1
    """

execute_sql(sql)

,week,weekday,City,Payment_Method,Order_State,count,rnk
0,00,Sunday,Accra,cash,delivered,98,1
1,00,Saturday,Accra,cash,delivered,83,2
2,00,Wednesday,Accra,cash,delivered,59,3
3,00,Thursday,Accra,cash,delivered,51,4
4,00,Friday,Accra,cash,delivered,31,5
5,00,Sunday,Accra,cashless,delivered,100,1
6,00,Saturday,Accra,cashless,delivered,86,2
7,00,Thursday,Accra,cashless,delivered,75,3
8,00,Wednesday,Accra,cashless,delivered,65,4
9,00,Friday,Accra,cashless,delivered,45,5


In [76]:
sql = """
WITH
  table1 AS(
  SELECT
    strftime('%W',
      created_date) week,
    CASE CAST (strftime('%w',
        created_date) AS integer)
      WHEN 0 THEN 'Sunday'
      WHEN 1 THEN 'Monday'
      WHEN 2 THEN 'Tuesday'
      WHEN 3 THEN 'Wednesday'
      WHEN 4 THEN 'Thursday'
      WHEN 5 THEN 'Friday'
    ELSE
    'Saturday'
  END
    AS weekday,
    City,
    Payment_Method,
    Order_State,
    COUNT(*) count
  FROM
    orders o
  JOIN
    payment p
  ON
    o.Order_ID = p.Order_ID
  WHERE
    Order_State = "delivered"
  GROUP BY
    week,
    weekday,
    City,
    Payment_Method),
  rnk_table AS (
  SELECT
    *,
    RANK() OVER (PARTITION BY city,week, Payment_Method ORDER BY count DESC ) AS rnk
  FROM
    table1 )
SELECT
  week,
  City,
  Payment_Method,
  weekday,
  count no_of_delivered_orders
FROM
  rnk_table
WHERE
  rnk = 1
ORDER BY
  week,
  City,
  Payment_Method,
  weekday 
  
"""

execute_sql(sql)

,week,City,Payment_Method,weekday,no_of_delivered_orders
0,00,Accra,cash,Sunday,98
1,00,Accra,cashless,Sunday,100
2,00,Lisbon,cashless,Friday,1110
3,01,Accra,cash,Saturday,87
4,01,Accra,cashless,Sunday,100
5,01,Lisbon,cashless,Thursday,1497
6,02,Accra,cash,Tuesday,94
7,02,Accra,cashless,Tuesday,117
8,02,Lisbon,cashless,Wednesday,1374
9,03,Accra,cash,Tuesday,120


 SELECT \
  case cast (strftime('%w', created_date) as integer)\
  when 0 then 'Sunday'\
  when 1 then 'Monday'\
  when 2 then 'Tuesday'\
  when 3 then 'Wednesday'\
  when 4 then 'Thursday'\
  when 5 then 'Friday'\
  else 'Saturday' end as weekday\
          FROM orders limit 5")
rows = c.fetchall()
rows